<h1><center>90</center></h1>

<h1><center>Про тестирование и<br>здравый смысл</center></h1>

<h1><center>Стас Рудаков</center><br><center><img src="datarobot.png" alt="DataRobot"></center></h1>

<h2><center>Minsk Python Meetup<br>Июль 2016</center></h2>

* 2013 - июнь: _Дескрипторы в теории и на практике_
* 2013 - август: _По-новому стреляем себе в ногу при работе с сетью_
* 2014 - январь: _Очередной скучный доклад про логгирование_
* 2014 - ноябрь: _Practical Python Packaging_

# Зачем Минску Python митап?

* для формирования сообщества
* для изучения нового и обмена опытом
* дает возможность научиться публично выступать
* дает возможность прославиться
* networking
* для поиска работы
* для создания образа классного места для Python разработчиков

![mole](mole.jpg)

# Зачем я пишу тесты

* так заведено среди программистов

* про это спрашивают на собеседованиях

# Квадрант тестирования (Brian Marick)

![quadrant](quadrant.png)

[Building Microservices](http://shop.oreilly.com/product/0636920033158.do), Sam Newman

# Пирамида тестирования (Mike Cohn)

![pyramid](pyramid.png)

* https://habrahabr.ru/post/266309/
* http://www.agilebuddha.com/agile/traditional-testing-will-be-dead-soon/

Риски:

* сложно разработать стратегию тестирования правильно
 - касается как технических решений
 - так и разделения ответственности/обязанностей/полномочий
* о стратегии тестирования задумываются, когда проект уже разросся
* бизнес видит лишь верхушку пирамиды
* сложно внедрить в уже написанный код

# На практике... пирамида еще раз

![pyramid](pyramid.png)

* web-сайт
* REST API
* Fabric

## Или лучше так


![pyramid](pyramid2.png)

[Building Microservices](http://shop.oreilly.com/product/0636920033158.do), Sam Newman

# Но в конце все равно будет так

![cupcake](cupcake.png)

[Introducing the Software Testing Cupcake (Anti-Pattern)](https://www.thoughtworks.com/insights/blog/introducing-software-testing-cupcake-anti-pattern)

# От теории к практике

In [ ]:
class TestAPI(unittest.TestCase):
    def test_search_users(self):
        client = app.test_client()
        self.assertEqual(client.get('/users/search/token').status_code, 403)
        client.login('admin@example.com', '111111')
        response = client.get('/users/search/user')
        self.assertEqual(response.status_code, 200)
        data = json.loads(response.data)
        self.assertItemsEqual(
            data['users'],
            [
                {'id': 17, 'username': 'user1@example.com'},
                {'id': 35, 'username': 'user2@example.com'},
                {'id': 68, 'username': 'user3@example.com'},
            ]
        )

# Что не так с этим тестом?

1. тестирует не одну вещь
2. непонятно, что вообще тестируется
3. код плохо отформатирован
4. последний список содержит `id`
5. непонятно, откуда берутся `admin@example.com`, `user*@example.com`
6. unittest-style assertAnything

# А что с ним так?

* ну... он вообще есть и тестирует реальный код

In [ ]:
# Проблема 1: Тестирует не одну вещь

-class TestAPI(unittest.TestCase):
-    def test_search_users(self):
+class TestSearchUsers(unittest.TestCase):
+    def test_permissions(self):
         client = app.test_client()
         self.assertEqual(client.get('/users/search/token').status_code, 403)
+
+    def test_response200(self):
+        client = app.test_client()
         client.login('admin@example.com', '111111')
         response = client.get('/users/search/user')
         self.assertEqual(response.status_code, 200)
         data = json.loads(response.data)
         self.assertItemsEqual(
             data['users'],
             [
                 {'id': 17, 'username': 'user1@example.com'},
                 {'id': 35, 'username': 'user2@example.com'},
                 {'id': 68, 'username': 'user3@example.com'},
             ]
         )

In [ ]:
# Проблема 2: непонятно, что вообще тестируется
# Проблема 3: код плохо отформатирован

 class TestSearchUsers(unittest.TestCase):
     def test_permissions(self):
         client = app.test_client()
-        self.assertEqual(client.get('/users/search/token').status_code, 403)
+
+        # when a user without admin permissions searches for users
+        response = client.get('/users/search/token')
+
+        # then response code should be 403 (forbidden)
+        self.assertEqual(response.status_code, 403)
 
     def test_response200(self):
         client = app.test_client()
         client.login('admin@example.com', '111111')
+
+        # when admin searches for users matching `user`
         response = client.get('/users/search/user')
+
+        # then response should have code 200 and contain 3 users
         self.assertEqual(response.status_code, 200)
-        data = json.loads(response.data)
         self.assertItemsEqual(
             data['users'],
             [
                 {'id': 17, 'username': 'user1@example.com'},
                 {'id': 35, 'username': 'user2@example.com'},
                 {'id': 68, 'username': 'user3@example.com'},
             ]
         )


## Given-When-Then: BDD

<pre>
<b>Story</b>: Returns go to stock

<b>In order to</b> keep track of stock
<b>As a</b> store owner
<b>I want to</b> add items back to stock when they're returned.

<b>Scenario 1:</b> Refunded items should be returned to stock
<b>Given</b> that a customer previously bought a black sweater from me
<b>And</b> I have three black sweaters in stock.
<b>When</b> he returns the black sweater for a refund
<b>Then</b> I should have four black sweaters in stock.

<b>Scenario 2:</b> Replaced items should be returned to stock
<b>Given</b> that a customer previously bought a blue garment from me
<b>And</b> I have two blue garments in stock
<b>And</b> three black garments in stock.
<b>When</b> he returns the blue garment for a replacement in black
<b>Then</b> I should have three blue garments in stock
<b>And</b> two black garments in stock.
</pre>

In [ ]:
class TestSearchUsers(unittest.TestCase):
    def test_non_admin(self):
        # given a user with permission can_search but w/o permission is_admin
        client = app.test_client()
        with mock.patch('User') as user_model_mock:
            user_model_mock.get_permissions.return_value = ['can_search']

            # when search users request is performed
            response = client.get('/users/search/token')

        # then response code should be 403 (forbidden)
        self.assertEqual(response.status_code, 403)

In [ ]:
# Проблема 4: последний список содержит `id`

 class TestSearchUsers(unittest.TestCase):
     def test_permissions(self):
         client = app.test_client()
 
         # when a user without admin permissions searches for users
         response = client.get('/users/search/token')
 
         # then response code should be 403 (forbidden)
         self.assertEqual(response.status_code, 403)
 
     def test_response200(self):
         client = app.test_client()
         client.login('admin@example.com', '111111')
 
         # when admin searches for users matching `user`
         response = client.get('/users/search/user')
 
         # then response should have code 200 and contain 3 users
         self.assertEqual(response.status_code, 200)
-        self.assertItemsEqual(
-            data['users'],
-            [
-                {'id': 17, 'username': 'user1@example.com'},
-                {'id': 35, 'username': 'user2@example.com'},
-                {'id': 68, 'username': 'user3@example.com'},
-            ]
+        self.assertEqual(
+            len(json.loads(response.data)['users']),
+            3
         )

In [ ]:
# Проблема 5: непонятно, откуда берутся `admin@example.com`, `user*@example.com`
# Проблема 6:  unittest-style assertAnything


-class TestSearchUsers(unittest.TestCase):
-    def test_permissions(self):
-        client = app.test_client()
+class TestSearchUser(object):
+    def _get(self, client, keyword):
+        url = '/users/search/%s' % keyword
+        return client.get(url)
 
+    def test_permissions(self, user_client):
         # when a user without admin permissions searches for users
-        response = client.get('/users/search/token')
+        response = self._get(user_client, 'token')
 
         # then response code should be 403 (forbidden)
-        self.assertEqual(response.status_code, 403)
+        assert response.status_code == 403
 
-    def test_response200(self):
-        client = app.test_client()
-        client.login('admin@example.com', '111111')
+    def test_response200(self, admin_client, user_factory):
+        # given 3 users with username contain token `user`
+        # and 1 user without `user` in username
+        user_factory(username='user1@example.com')
+        user_factory(username='user2@example.com')
+        user_factory(username='user3@example.com')
+        user_factory(username='token1@example.com')
 
         # when admin searches for users matching `user`
-        response = client.get('/users/search/user')
+        response = self._get(admin_client, 'user')
 
         # then response should have code 200 and contain 3 users
-        self.assertEqual(response.status_code, 200)
-        self.assertEqual(
-            len(json.loads(response.data)['users']),
-            3
-        )
+        assert response.status_code == 200
+        assert len(json.loads(response.data)['users']) == 3


# [pytest](http://pytest.org) от Holger Krekel и команды

Guido van Rossum, [bugs.python.org](https://bugs.python.org/issue17908):
> Not sure if the default test runner is something to extend at this point. Eveybody is using py.test anyway...

["Нескучное тестирование с pytest" - Роман Иманкулов](http://www.slideshare.net/imankulov/pytest-testing)

```
$ cat test_package/test_module.py
def test_function():
    assert True


class TestClass(object):
    def test_method(self):
        assert list(range(5)) == list(range(6))
```
```
$ py.test -v
======================= test session starts =======================
collected 2 items 

test_package/test_module.py::test_function PASSED
test_package/test_module.py::TestClass::test_method FAILED

============================ FAILURES =============================
______________________ TestClass.test_method ______________________

self = <test_module.TestClass object at 0x7f6213ddceb8>

    def test_method(self):
>       assert list(range(5)) == list(range(6))
E       assert [0, 1, 2, 3, 4] == [0, 1, 2, 3, 4, 5]
E         Right contains more items, first extra item: 5
E         Full diff:
E         - [0, 1, 2, 3, 4]
E         + [0, 1, 2, 3, 4, 5]
E         ?               +++

test_package/test_module.py:7: AssertionError
=============== 1 failed, 1 passed in 0.01 seconds ================
```

# pytest: явные зависимости

In [ ]:
@pytest.fixture
def user_client():
    return app.test_client()

@pytest.fixture
def admin_client(user_client, admin):
    user_client.login(admin.username, admin.password)
    return user_client

class TestSearchUser(object):
    # ...

    def test_permissions(self, user_client):
        # ...
        user_client.get('/users/search/token')
        # ...
    
    def test_response200(self, admin_client, user_factory):
        user_factory(username='user1@example.com')
        user_factory(username='user2@example.com')
        user_factory(username='user3@example.com')
        user_factory(username='token1@example.com')
        # ...
        admin_client.get('/users/search/user')
        # ...

# assert и pdb

```
$ py.test -v --pdb
[...]
test_package/test_module.py::TestClass::test_method FAILED
>>>>>>>>>>>>>>>>>>>>>>>>>>>> traceback >>>>>>>>>>>>>>>>>>>>>>>>>>>>
self = <test_module.TestClass object at 0x7f43cdf44898>

    def test_method(self):
>       assert list(range(5)) == list(range(6))
E       assert [0, 1, 2, 3, 4] == [0, 1, 2, 3, 4, 5]
E         Right contains more items, first extra item: 5
E         Full diff:
E         - [0, 1, 2, 3, 4]
E         + [0, 1, 2, 3, 4, 5]
E         ?               +++

test_package/test_module.py:7: AssertionError
>>>>>>>>>>>>>>>>>>>>>>>>>> entering PDB >>>>>>>>>>>>>>>>>>>>>>>>>>>
> /tmp/xxx/test_package/test_module.py(7)test_method()
-> assert list(range(5)) == list(range(6))
(Pdb) 
```

In [ ]:
class TestSearchUser(object):
    def _get(self, client, keyword):
        url = '/users/search/%s' % keyword
        return client.get(url)

    def test_permissions(self, user_client):
        # when a user without admin permissions searches for users
        response = self._get(user_client, 'token')

        # then response code should be 403 (forbidden)
        assert response.status_code == 403

    def test_response200(self, admin_client, user_factory):
        # given 3 users with username contain token `user`
        # and 1 user without `user` in username
        user_factory(username='user1@example.com')
        user_factory(username='user2@example.com')
        user_factory(username='user3@example.com')
        user_factory(username='token1@example.com')

        # when admin searches for users matching `user`
        response = self._get(admin_client, 'user')

        # then response should have code 200 and contain 3 users
        assert response.status_code == 200
        assert len(json.loads(response.data)['users']) == 3

# Тесты для кода или код для тестов?


In [ ]:
def get_python_org():
    response = requests.get('http://python.org')
    if response.status_code == 200:
        return response.content
    return None

In [ ]:
def test_python_org_404():
    response = requests.Response()
    response.status_code = 404
    with mock.patch('requests.get') as get_mock:
        get_mock.return_value = response
        
    result = get_python_org()
    
    assert result is None

In [ ]:
from requests import get

def get_python_org():
    response = get('http://python.org')
    if response.status_code == 200:
        return response.content
    return None

In [ ]:
def get_python_org(requests=requests):
    response = requests.get('http://python.org')
    if response.status_code == 200:
        return response.content
    return None

In [ ]:
def test_python_org_404():
    response = requests.Response()
    response.status_code = 404
    requests_mock = Mock(**{'get.return_value': response})
        
    result = get_python_org(requests=requests_mock)
    
    assert result is None

# Окружение для тестов

* готовим внешние зависимости для тестов (сервисы, базы данных) снаружи тестов

* запихиваем наши тесты в Docker (или любые другие) контейнеры или виртуальные машины

* запускаем зависимости в виде Docker контейнеров изнутри тестов

## Все зависит от доступных инструментов

In [ ]:
@pytest.yield_fixture(scope="session")
def postgres_port():
    from docker import Client
    docker_client = Client(base_url=DOCKER_URL)
    docker_client.pull(PG_IMAGE)
    # creating and starting container
    container = docker_client.create_container(image=PG_IMAGE)
    container_id = container['Id']
    docker_client.start(container=container_id, port_bindings={'5432/tcp': 0})
    cdata = docker_client.inspect_container(container_id)
    # getting real value for randomized port (0 in port_bindings)
    pg_port = int(cdata['NetworkSettings']['Ports']['5432/tcp'][0]['HostPort'])
    # waiting for image to start
    for _ in range(60):
        logs = docker_client.logs(container_id).decode('utf-8')
        if 'database system is ready to accept connections' in logs:
            break
        else:
            time.sleep(1)
    else:
        docker_client.kill(container=container_id)
        docker_client.remove_container(container=container_id, v=True)
        raise Exception('Postgres Docker image didn\'t start in 60 seconds')

    # returning control back to py.test
    yield pg_port

    # stopping and removing container
    docker_client.stop(container=container_id)
    docker_client.remove_container(container=container_id, v=True)

In [ ]:
def test_select_1(postgres_port):
    import psycopg2
    conn = psycopg2.connect(host='127.0.0.1', port=postgres_port,
                            user='postgres', password='',
                            dbname='postgres')
    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    cursor.fetchall()

# Вместо вывода

## Интересные темы

* Python как первый язык программирования
* Python как новый язык программирования
* Hypothesis - property based testing
* asyncio
* Wagtail CMS
* Jupyter
* Написание расширений на языке Rust
* PySpark
* PyStorm

# Стас Рудаков
<pre>mailto:stas@garage22.net</pre>
[github.com/nott/talks/tree/master/testing](https://github.com/nott/talks/tree/master/testing)